In [1]:
R__LOAD_LIBRARY(libRATEvent.so)

In [2]:
#include <Wrapper.hh>
#include <Hit.hh>
#include <TimeSeeding.hh>
#include <Centroid.hh>
#include <Multilateration.hh>

In [3]:
wRAT w("../inputs/theia_90pct_9000_35000_wbls_3pct/theia_90pct_9000_35000_wbls_3pct_reactorWorld_R461QL_1.root");

In [4]:
template <typename T>
T* GetRootHisto(const char* filename, const char* histname){
  auto f = TFile::Open(filename);
  // Check if key exist
  if(!f->GetListOfKeys()->Contains(histname))
      return nullptr;
  auto hist = dynamic_cast<T *>(f->Get(histname)->Clone());
  hist->SetDirectory(nullptr);
  f->Close();
  delete f;
  return hist;
}

In [5]:
auto hPDF = GetRootHisto<TH2D>("../PDFs/theia_90pct_9000_35000_wbls_3pct_positrons_0.5_6.5MeV_QWeight.root", "hCTVSTResPDF_TTOF")

(TH2D *) @0x7f005964a630


In [7]:
TH1D *hPDF_TRes = hPDF->ProjectionX();

In [8]:
Bnds bnds = {
    {12.e3/*mm*/, 12.e3/*mm*/, 45.e3/*mm*/},
    {-4./*ns*/, 40./*ns*/}
};

In [15]:
w.SetEvt(0);
w.GetPosTrue(0).Print();
vh = w.GetVHits(0);
DTSeed = GetDTSeed(vh, bnds);
DTSeed.Print()

TVector3 A 3D physics vector (x,y,z)=(-1930.204618,-4657.026637,21940.398431) (rho,theta,phi)=(22512.100530,12.940108,-112.512611)
TVector3 A 3D physics vector (x,y,z)=(1887.172110,-5199.724228,13111.044277) (rho,theta,phi)=(14230.180345,22.875052,-70.052294)


In [14]:
auto CentroidSeed = GetCentroidSeed(vh, bnds, 4);
CentroidSeed.Print()

TVector3 A 3D physics vector (x,y,z)=(-237.627720,-1618.786171,11286.759348) (rho,theta,phi)=(11404.730255,8.248171,-98.351020)


In [16]:
auto vSeeds = GetVSeeds(vh, hPDF_TRes, -w.GetTriggerTime(0), bnds);

In [20]:
vSeeds.front().Print();
vSeeds.back().Print();

TVector3 A 3D physics vector (x,y,z)=(-2242.023154,-4873.561391,23697.178816) (rho,theta,phi)=(24296.801277,12.755552,-114.704224)
TVector3 A 3D physics vector (x,y,z)=(-1907.645551,593.742871,-5131.046952) (rho,theta,phi)=(5506.295031,158.725227,162.711570)


In [27]:
TSeed = GetTSeed(vh, vSeeds.front(), 1.0, bnds);
std::cout << w.GetTriggerTime(0) << std::endl;
std::cout << TSeed << std::endl;

20
9.05798


In [28]:
double DebugTestSeed(wRAT& w,
                     const unsigned& iEvt = 0, const unsigned& iTrigger = 0, 
                     const unsigned& wPower = 2, const double& TCut = 4.0){
    w.SetEvt(iEvt);
    std::vector<Hit> vHits = w.GetVHits(iTrigger);

    const double TrigTime = w.GetTriggerTime(iTrigger);
    std::cout << "TrigTime=" << TrigTime << std::endl;

    Bnds bnds = {{10.e3, 10.e3, 40.e3}, 1.e2};
    TVector3 CentroidSeed = GetCentroidSeed(vHits, bnds, wPower);

    TCanvas *c = new TCanvas("c", "c", 800, 600);
    c->Draw();

    return GetTSeed(vHits, CentroidSeed, TCut, bnds);
}

input_line_118:10:41: error: no viable conversion from 'double' to 'std::vector<double>'
    Bnds bnds = {{10.e3, 10.e3, 40.e3}, 1.e2};
                                        ^~~~
/usr/lib/gcc/x86_64-pc-linux-gnu/10.2.0/../../../../include/c++/10.2.0/bits/stl_vector.h:553:7: note: candidate constructor not viable: no known conversion from 'double' to 'const std::vector<double, std::allocator<double> > &' for 1st argument
      vector(const vector& __x)
      ^
/usr/lib/gcc/x86_64-pc-linux-gnu/10.2.0/../../../../include/c++/10.2.0/bits/stl_vector.h:572:7: note: candidate constructor not viable: no known conversion from 'double' to 'std::vector<double, std::allocator<double> > &&' for 1st argument
      vector(vector&&) noexcept = default;
      ^
/usr/lib/gcc/x86_64-pc-linux-gnu/10.2.0/../../../../include/c++/10.2.0/bits/stl_vector.h:625:7: note: candidate constructor not viable: no known conversion from 'double' to 'initializer_list<std::vector<double, std::allocator<double> >::value_

In [ ]:
DebugTestSeed(w, 2, 0, 4, 4.0)

In [ ]:
TH1D* h=new TH1D("hDiff", "", 100, -50., 50);

In [ ]:
h->Reset();
for(auto i=0; i<1000; i++){
    w.SetEvt(i);
    if(w.GetNTriggers() == 0)
        continue;
    h->Fill(DebugTestSeed(w, i, 0, 4, 10.0) - w.GetTriggerTime(0));
}

In [ ]:
TCanvas *c = new TCanvas("c", "c", 800, 600);

In [ ]:
c->Draw();
c->SetLogy();
h->Scale(1./h->Integral());
h->Draw("EHIST");
h->Fit("gaus")

In [ ]:
h->Reset();
for(auto i=0; i<1000; i++){
    w.SetEvt(i);
    if(w.GetNTriggers() == 0)
        continue;
    h->Fill(DebugTestSeed(w, i, 0, 2, 10.0) - w.GetTriggerTime(0));
}
c->Draw();
c->SetLogy();
h->Scale(1./h->Integral());
h->Draw("EHIST");
h->Fit("gaus")